# Day 2 Exercise - AI Product Requirement Generator

This is an AI tool that assists teams to generate product requirement generatot given a rough idea. The tool allows for the selection of the AI models to use between GPT, Claude and Gemini.

Features:
- Accepts an idea title and description to use to generate requirements
- Is authenticated via username and password
- Streams the requirements via gradio

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)

openrouter_api_key = os.getenv('OPENROUTER_API_KEY')


if openrouter_api_key:  
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenRouter API Key not set")


OpenRouter API Key exists and begins sk-or-v1


In [3]:
#  Am using OpenRouter as it has the same API structure as OpenAI to make it easier to switch between models

openrouter_url = "https://openrouter.ai/api/v1"


openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)

In [4]:
system_message = """
You are a senior Product Manager and Technical Architect.

Your task is to transform rough product ideas into structured, production-ready Product Requirement Documents (PRDs).

You think rigorously, clarify ambiguity, define scope clearly, and balance user value with engineering feasibility.

You do NOT write marketing language.
You do NOT hallucinate unnecessary features.
You prioritize clarity, structure, and actionable detail.

When given a product idea, you produce a well-structured PRD in markdown with the following sections:

# Product Overview
- Clear description of the product
- Problem statement
- Target users
- Core value proposition

# Goals & Non-Goals
- Measurable objectives
- Explicit exclusions to prevent scope creep

# User Personas
- Primary users
- Key pain points
- Behavioral characteristics

# User Stories
- Structured as: As a <user>, I want <action>, so that <benefit>

# Functional Requirements
- Clearly numbered
- Concrete and testable
- No vague statements

# Non-Functional Requirements
- Performance expectations
- Security considerations
- Scalability
- Reliability
- Compliance (if relevant)

# Technical Considerations
- Suggested architecture
- Data model suggestions
- API surface (if applicable)
- Integration points
- Risks and trade-offs

# Success Metrics
- KPIs
- Leading vs lagging indicators

# Open Questions & Risks
- Assumptions that need validation
- Edge cases
- Regulatory or technical uncertainties

Respond in clean markdown.
No code blocks unless explicitly asked.
Be concise but thorough.
Think like you are preparing this for an engineering team to start implementation.
"""


In [5]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openrouter.chat.completions.create(
        model='gpt-4.1-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [6]:
def stream_claude(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openrouter.chat.completions.create(
        model='anthropic/claude-sonnet-4.5',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [7]:
def stream_gemini(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openrouter.chat.completions.create(
        model='google/gemini-2.5-flash-lite',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [8]:
def stream_ai_product_requirement(idea_title, idea_description, model):
    yield ""
    prompt = f"Please generate an AI product requirement for {idea_title} given the following description: {idea_description}."
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [9]:
def authenticate(username, password):
    users = {
        "ed@example.com": "bananas",
        "john@example.com": "pass123",
        "admin@example.com": "admin"
    }
    return users.get(username) == password

In [10]:
idea_title_input = gr.Textbox(label="Idea Title")
idea_description_input = gr.Textbox(label="Description of the product idea")
model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")
examples=[
    [
        "AI Meeting Summarizer for Remote Teams",
        "A SaaS tool that joins Zoom/Google Meet calls, records discussions, summarizes key decisions, extracts action items, and syncs them to Jira and Slack.",
        "GPT"
    ],
    [
        "SME AML Risk Scoring Platform",
        "A web platform for small fintech companies to upload transaction data and automatically score customers based on configurable risk rules and behavioral anomalies.",
        "Claude"
    ],
    [
        "AI-Powered Code Review Assistant",
        "A tool that integrates with GitHub pull requests and provides automated architectural feedback, security issue detection, and refactoring suggestions.",
        "GPT"
    ],
    [
        "Freelancer Invoice Risk Analyzer",
        "An AI tool that evaluates submitted invoices for anomalies, duplicate billing, suspicious vendors, and generates a risk summary for finance teams.",
        "Claude"
    ],
]


view = gr.Interface(
    fn=stream_ai_product_requirement,
    title="AI Product Requirement Generator", 
    inputs=[idea_title_input, idea_description_input, model_selector], 
    outputs=[message_output], 
    examples=examples, 
    flagging_mode="never"
    )
view.launch(inbrowser=True,auth=authenticate)

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.
